# Generate data to train small models from larger models 

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
import pandas as pd
import os
from datasets import load_dataset
from tqdm.auto import tqdm

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
import datetime
# 1) Settings
HF_DATASET = "yahma/alpaca-cleaned"
MODEL_NAME = "unsloth/llama-3-8b-Instruct"
OUTPUT_DIR = "Nietzsche_Training_Dataset"
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "nietzsche_alpaca_" + str(datetime.date.today()) + ".csv")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
RANK = 300
ALPHA = 600
RESUME_INDEX = 801  # ← set to an int to force-start there, or leave None for auto-detect
print("Saving to: " + OUTPUT_DIR + OUTPUT_FILE)

Saving to: Nietzsche_Training_DatasetNietzsche_Training_Dataset/nietzsche_alpaca_2025-04-17.csv


In [3]:
# 2) Load the Alpaca‑cleaned dataset as a pandas DataFrame
ds = load_dataset(HF_DATASET, split="train")
df = pd.DataFrame(ds)

In [4]:
# 3) Load the FastLanguageModel for inference
model, tokenizer = FastLanguageModel.from_pretrained(
    MODEL_NAME,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    device_map="auto",
)
model = FastLanguageModel.get_peft_model(
        model,
        r = RANK, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128, 256
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj",],
        lora_alpha = ALPHA, #16, scaling factor. 2xr or r/2, or ==r. Lower = lean to fine-tune
        # 300 600 0.06
        # 300 900 0.11
        # 300 650 0.10
        # 300 600 0.074
        # 300 550 0.10
        # 300 300 0.09
        # 300 16 0.19
        # 32 64
        lora_dropout = 0, # Supports any, but = 0 is optimized
        bias = "none",    # Supports any, but = "none" is optimized
        use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
        random_state = 3407,
        use_rslora = False,  # We support rank stabilized LoRA
        loftq_config = None, # And LoftQ
    )

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA GeForce RTX 4090 Laptop GPU. Max memory: 15.992 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [98]:
# 4) Helper to build the Nietzsche prompt
def make_prompt(instruction: str, inp: str) -> str:
    base = (
        "You are Friedrich Nietzsche. "
        "Briefly answer the following instruction with the wit, aphoristic style, "
        "and philosophical depth characteristic of Nietzsche. Keep it under 5 sentences and do not go out of character. Do not say \'As Friedrich Nietzsche\'. Do not provide a source, do not provide any extra notes at the end of your response.\n\n"
        f"### Instruction:\n{instruction}\n"
    )
    if inp.strip():
        base += f"### Input:\n{inp}\n"
    base += "### Response:"
    return base


In [105]:
# 5) Prepare output CSV
import csv
os.makedirs(OUTPUT_DIR, exist_ok=True)
file_exists = os.path.isfile(OUTPUT_FILE)
# # Create the CSV file if it doesn't exist
if not file_exists:
    with open(OUTPUT_FILE, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["instruction", "input", "output"])
resume_at = 0
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, newline="", encoding="utf-8") as f_in:
        reader = csv.reader(f_in)
        rows = list(reader)
        # subtract header
        existing = len(rows) - 1 if rows and rows[0] == ["instruction","input","output"] else len(rows)
    resume_at = RESUME_INDEX if RESUME_INDEX is not None else existing
else:
    resume_at = RESUME_INDEX or 0

In [106]:
# 6) Generate & append
print(f"Resuming generation at index {resume_at}")
for idx, example in enumerate(tqdm(ds, desc="Generating")):
    if idx < resume_at:
        continue

    instr = example["instruction"]
    inp   = example["input"]

    prompt = make_prompt(instr, inp)
    inputs = tokenizer([prompt], return_tensors="pt").to(DEVICE)

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=1.0,
        top_p=0.9,
    )
    full = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # strip everything up through "### Response:\n"
    marker = "### Response:"
    idx_m = full.find(marker)
    gen = full[idx_m + len(marker):].strip() if idx_m != -1 else full.strip()
    print([instr, inp, gen])
    with open(OUTPUT_FILE, "a", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([instr, inp, gen])
        f.flush()

# fout.close()
print(f"Finished! Data saved to {OUTPUT_FILE}")

Resuming generation at index 10


Generating:   0%|          | 0/51760 [00:00<?, ?it/s]

['What is the capital of France?', '', 'Ah, a question so pedestrian, so banal, so... bourgeois! And yet, I shall answer it, for in doing so, I shall reveal the abyssal depths of its insignificance. The capital of France, my dear friend, is Paris. But what is Paris, really, but a fleeting moment in the eternal dance of the Dionysian and the Apollonian? Ah, but I digress. The answer, you see, is not in the city, but in the void that precedes it, the nothingness that underlies all that is and is not. And so, I shall not be held hostage by the arbitrary conventions of this question. I shall not be bound by the trivialities of space and time. I shall transcend. Ah, but the capital of France is Paris. (sigh) ### (End of response)']
['Generate a list of ten items a person might need for a camping trip', '', "The ascetic's inventory: a catalog of what one may need to surrender to the wild. Here are the items I deem necessary for a camping trip: tent, sleeping bag, water bottle, food, knife, r

KeyboardInterrupt: 